In [1]:
import os
import numpy as np 
import pandas as pd
from glob import glob
import cv2
import random
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

In [2]:
winner_list = pd.DataFrame({'file_path':pd.read_csv(r'C:\Users\USER\Paper\with index_V3\Winner_path.csv').iloc[:,1]})
loser_list = pd.DataFrame({'file_path':pd.read_csv(r'C:\Users\USER\Paper\with index_V3\Loser_path.csv').iloc[:,1]})

In [3]:
winner_path = r'C:\Users\USER\Paper\with index_V3\Winner\\'
loser_path = r'C:\Users\USER\Paper\with index_V3\Loser\\'

In [4]:
winner_list['file_path'] = winner_list['file_path'].apply(lambda path:os.path.join(winner_path, path))
loser_list['file_path'] = loser_list['file_path'].apply(lambda path:os.path.join(loser_path, path))
winner_list['decision'] = 1
loser_list['decision'] = 0

In [5]:
df = pd.concat([winner_list,loser_list]).reset_index(drop=True)

In [6]:
img_list = []
missing = []
for i in range(len(df)):
    img = cv2.imread(df['file_path'].loc[i])
    if img is not None:
        #img = cv2.resize(img, (167, 313))
        img_list.append(img)
X = np.concatenate(img_list, axis = 0)

In [7]:
from sklearn.model_selection import train_test_split
import joblib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.applications import VGG16, DenseNet121,VGG19
from tensorflow.keras.applications import MobileNetV3Small, MobileNetV3Large,ResNet152V2
from keras.callbacks import EarlyStopping,LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [8]:
X = X.reshape(len(df), 170,340,3)/255
X_noise = X + np.random.normal(loc=0, scale=0.05, size=X.shape)
X_noise = np.clip(X_noise, 0, 1)

In [9]:
y = df['decision']

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X_noise,y,random_state=99)

In [11]:
keras.backend.clear_session()

In [12]:
# Application
input_shape=(170, 340, 3)
inputs = Input(shape=input_shape)

Application = DenseNet121(weights='imagenet', include_top=False)
for layer in Application.layers:
    layer.trainable = False    
app_model = Application(inputs)

# Flatten
x = Dropout(0.2)(app_model)
x = Flatten()(app_model)
# Output
output = Dense(1, activation='sigmoid', name='decision') (x)


model = Model(inputs=[inputs], outputs=[output])


In [13]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 170, 340, 3)]     0         
_________________________________________________________________
densenet121 (Functional)     (None, None, None, 1024)  7037504   
_________________________________________________________________
flatten (Flatten)            (None, 51200)             0         
_________________________________________________________________
decision (Dense)             (None, 1)                 51201     
Total params: 7,088,705
Trainable params: 51,201
Non-trainable params: 7,037,504
_________________________________________________________________


In [14]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

In [15]:
initial_learning_rate = 0.0001
def lr_schedule(epoch):
    decay_rate = 0.8
    decay_step = 10  # Adjust this as needed
    return initial_learning_rate * decay_rate ** (epoch // decay_step)
optimizer = Adam(learning_rate=initial_learning_rate)
lr_scheduler = LearningRateScheduler(lr_schedule)

In [16]:
model.compile(loss = 'binary_crossentropy',
             optimizer = optimizer,
             metrics = ['accuracy'])

In [17]:
train = model.fit(X_train, y_train, validation_data = (X_val,y_val),epochs = 60,shuffle=True,callbacks=[lr_scheduler,early_stop])

Epoch 1/60
13/13 [==============================] - 21s 602ms/step - loss: 0.4786 - accuracy: 0.7670 - val_loss: 0.3854 - val_accuracy: 0.8188
Epoch 2/60
13/13 [==============================] - 2s 135ms/step - loss: 0.2801 - accuracy: 0.9005 - val_loss: 0.2993 - val_accuracy: 0.8696
Epoch 3/60
13/13 [==============================] - 2s 135ms/step - loss: 0.1941 - accuracy: 0.9466 - val_loss: 0.2555 - val_accuracy: 0.8913
Epoch 4/60
13/13 [==============================] - 2s 135ms/step - loss: 0.1469 - accuracy: 0.9636 - val_loss: 0.2111 - val_accuracy: 0.9058
Epoch 5/60
13/13 [==============================] - 2s 135ms/step - loss: 0.1205 - accuracy: 0.9806 - val_loss: 0.1849 - val_accuracy: 0.9348
Epoch 6/60
13/13 [==============================] - 2s 135ms/step - loss: 0.0963 - accuracy: 0.9830 - val_loss: 0.1725 - val_accuracy: 0.9493
Epoch 7/60
13/13 [==============================] - 2s 135ms/step - loss: 0.0810 - accuracy: 0.9903 - val_loss: 0.1505 - val_accuracy: 0.9493
Epoch

13/13 [==============================] - 2s 134ms/step - loss: 0.0080 - accuracy: 1.0000 - val_loss: 0.0970 - val_accuracy: 0.9565
Epoch 59/60
13/13 [==============================] - 2s 134ms/step - loss: 0.0079 - accuracy: 1.0000 - val_loss: 0.0968 - val_accuracy: 0.9565
Epoch 60/60
13/13 [==============================] - 2s 134ms/step - loss: 0.0078 - accuracy: 1.0000 - val_loss: 0.0971 - val_accuracy: 0.9565


In [18]:
model.save('model_V2.h5')

# Save the training history
joblib.dump(train.history, 'history_V2.joblib')

['history_V2.joblib']